In [1]:
# Import the game
import gym_super_mario_bros
# Import the joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

Creo mi environment usando gym_super_mario_bros, esta libreria emula el Super Mario Bros.
Ademas usare un entorno de actions reducido con SIMPLE_MOVEMENT

Mi idea es entrenar una AI con el primer nivel de Mario y comprobar su rendimiento posterior en niveles aleatorios.

In [2]:
# Codigo para el test
# env_test.seed(42)
# env_test = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', stages=['1-2', '1-3', '1-4', '2-1'])

In [3]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [4]:
env.action_space

Discrete(7)

In [5]:
# Este codigo ejecuta una accion aleatoriamente, codigo de prueba
done = True

for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [6]:
# Mi idea es crear un wrapper mas adelante para modificar mi funcion de reward y anadir recompensas por coger monedas y setas para subir el nivel
info

{'coins': 0,
 'flag_get': False,
 'life': 2,
 'score': 0,
 'stage': 1,
 'status': 'small',
 'time': 150,
 'world': 1,
 'x_pos': 594,
 'x_pos_screen': 79,
 'y_pos': 103}

In [7]:
# Con estos wrappers hare modificaciones a mi imagen para prepararlas para mi AI
from gym.wrappers import GrayScaleObservation, ResizeObservation

# Importo los dos tipos de vectorizadores para mi entorno
# Primero usare DummyVecEnv para conseguir mi primer entrenamiento estable, una vez lo obtenga, pasare a usar SubprocVecEnv para optimizarlo.
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecFrameStack
# Usare el framestack de stable baselines ya que vectoriza directamente

# Tambien importare el modelo PPO de stable baselines. De esta manera usare un algoritmo ya implementado para configurar mis rewards y mi entorno
from stable_baselines3 import PPO 

In [12]:
# Creo una funcion para inicializar mis env en DummyVecEnv
def make_env():
    # Inicializo mi env y le paso los comandos de action
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    # Preparo el preprocesamiento de la imagen
    env = GrayScaleObservation(env, keep_dim=True) # Keep dim en True para mantener las dimensiones
    env = ResizeObservation(env, shape=(84, 89)) # Esta resolucion es la utilizada normalmente, aunque cambia la relacion de aspecto, solo lo hace ligeramente
    return env

In [13]:
# Compruebo el formato de mi output para ver que todo este correcto
state.shape

(240, 256, 3)

In [14]:
env = DummyVecEnv([make_env for _ in range(8)]) # Con esto creare 8 env simultaneos para acelerar el proceso

env = VecFrameStack(env, 4, channels_order='last') # Numero de frames que guardare para tener contexto

In [ ]:
LOG_DIR = './logs/' # Carpeta donde guardare los tensorboard logs

model = PPO("CnnPolicy", env=env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
# TODO Añadir Callback para hacer seguimiento de mi entrenamiento